## Librerías

In [9]:
%%capture
%run "../../recurrentes.ipynb"

In [10]:
%%capture
%run "../../funciones.ipynb"

In [11]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:
%config SqlMagic.named_parameters = "enabled"



The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Creación clase ternaria

In [12]:
# %%sql

# create or replace table competencia_03 as
# with periodos as (
#     select distinct foto_mes from competencia_03_crudo
# ), clientes as (
#     select distinct numero_de_cliente from competencia_03_crudo
# ), todo as (
#     select numero_de_cliente, foto_mes from clientes cross join periodos
# ), clase_ternaria as (
#     select
#         c.*
#         , if(c.numero_de_cliente is null, 0, 1) as mes_0 
#         , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
#         , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
#         , if(mes_2 = 1, 'CONTINUA',
#              if(mes_1 = 0, 'BAJA+1',
#                 if(mes_2 = 0,'BAJA+2',NULL))) as clase_ternaria 
#     from todo t
#     left join competencia_03_crudo c using (numero_de_cliente, foto_mes)
# ) 

# select * EXCLUDE (mes_0, mes_1, mes_2) 
# from clase_ternaria
# where mes_0 = 1 
# ;


In [13]:
# %%sql 
# SELECT * FROM competencia_03;

In [14]:
# %sql COPY competencia_03 TO 'D:/competencia_03.parquet' (FORMAT 'PARQUET');

# Feature Engineering

In [15]:
%%sql
CREATE OR REPLACE TABLE competencia_03 AS
SELECT * FROM read_parquet(:dataset_clase_ternaria_sin_psi_l);

,Success


In [16]:
%%sql 
SELECT * FROM competencia_03;

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,cliente_edad,cliente_antiguedad,mrentabilidad,mactivos_margen,mpasivos_margen,cproductos,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,250480925,201901,1,0,39,166,2734.28,-221.90,1424.55,7,...,1.0,0.0,-23237.35,0.0,2177.0,6766.59,9.0,0.0,938.40,CONTINUA
1,250480925,201902,1,0,39,167,2269.95,558.15,685.30,7,...,1.0,0.0,-11730.00,0.0,2205.0,6748.23,12.0,0.0,1630.47,CONTINUA
2,250480925,201903,1,0,39,168,3928.99,897.46,1584.45,7,...,4.0,10557.0,-25731.12,0.0,2236.0,13258.06,18.0,0.0,0.00,CONTINUA
3,250480925,201904,1,0,39,169,3431.69,247.72,1537.30,7,...,-1.0,0.0,-19107.25,0.0,2266.0,17332.37,16.0,0.0,1243.38,CONTINUA
4,250480925,201905,1,0,39,170,0.00,0.00,0.00,7,...,2.0,0.0,-14662.50,0.0,2297.0,8311.09,16.0,0.0,2557.14,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4901232,1590099528,202105,1,0,55,1,134.97,2.63,112.44,7,...,5.0,0.0,NaN,NaN,28.0,NaN,NaN,NaN,0.00,CONTINUA
4901233,1590099528,202106,1,0,55,2,228.19,3.45,19.10,7,...,0.0,0.0,NaN,NaN,58.0,NaN,NaN,NaN,0.00,CONTINUA
4901234,1590099528,202107,1,0,55,3,318.34,1.23,98.19,7,...,3.0,0.0,NaN,NaN,89.0,NaN,NaN,NaN,0.00,CONTINUA
4901235,1590099528,202108,1,0,55,4,469.97,58.16,154.16,7,...,6.0,0.0,NaN,NaN,120.0,NaN,NaN,NaN,0.00,None


In [17]:
paired_variables = [
    ('mtarjeta_visa_consumo', 'mtarjeta_master_consumo'),
    ('ctarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos'),
    ('mttarjeta_visa_debitos_automaticos', 'mttarjeta_master_debitos_automaticos'),
    ('ctarjeta_visa_descuentos', 'ctarjeta_master_descuentos'),
    ('mtarjeta_visa_descuentos', 'mtarjeta_master_descuentos'),
    ('Visa_Fvencimiento', 'Master_Fvencimiento'),
    ('Visa_msaldototal', 'Master_msaldototal'),
    ('Visa_msaldopesos', 'Master_msaldopesos'),
    ('Visa_msaldodolares', 'Master_msaldodolares'),
    ('Visa_mconsumospesos', 'Master_mconsumospesos'),
    ('Visa_mconsumosdolares', 'Master_mconsumosdolares'),
    ('Visa_mlimitecompra', 'Master_mlimitecompra'),
    ('Visa_madelantopesos', 'Master_madelantopesos'),
    ('Visa_madelantodolares', 'Master_madelantodolares'),
    ('Visa_fultimo_cierre', 'Master_fultimo_cierre'),
    ('Visa_mpagado', 'Master_mpagado'),
    ('Visa_mpagospesos', 'Master_mpagospesos'),
    ('Visa_mpagosdolares', 'Master_mpagosdolares'),
    ('Visa_fechaalta', 'Master_fechaalta'),
    ('Visa_mconsumotal', 'Master_mconsumototal'),
    ('Visa_cconsumos', 'Master_cconsumos'),
    ('Visa_cadelantosefectivo', 'Master_cadelantosefectivo'),
    ('Visa_mpagominimo', 'Master_mpagominimo')
]

In [18]:
# lista_feature = ['active_quarter','mrentabilidad','mrentabilidad_annual','mcuenta_corriente','mcaja_ahorro','mcuentas_saldo','ctarjeta_debito_transacciones','mautoservicio','ctarjeta_visa_transacciones','mtarjeta_visa_consumo','mtarjeta_master_consumo','cprestamos_personales','mprestamos_prendarios','cprestamos_hipotecarios','mprestamos_hipotecarios','cplazo_fijo','mplazo_fijo_dolares','mplazo_fijo_pesos','cpayroll_trx','mpayroll','mpayroll2','cpayroll2_trx','ccuenta_debitos_automaticos','mcuenta_debitos_automaticos','ctarjeta_visa_debitos_automaticos','mttarjeta_visa_debitos_automaticos','ctarjeta_master_debitos_automaticos','mttarjeta_master_debitos_automaticos','cpagodeservicios','mpagodeservicios','mpagomiscuentas','cpagomiscuentas','ccajeros_propios_descuentos','mcajeros_propios_descuentos','ctarjeta_visa_descuentos','mtarjeta_visa_descuentos','ctarjeta_master_descuentos','mtarjeta_master_descuentos','ccomisiones_mantenimiento','mcomisiones_mantenimiento','ccomisiones_otras','mcomisiones_otras','cforex','cforex_buy','mforex_buy','cforex_sell','mforex_sell','ctransferencias_recibidas','mtransferencias_recibidas','ctransferencias_emitidas','mtransferencias_emitidas','cextraccion_autoservicio','mextraccion_autoservicio','ccheques_depositados','mcheques_depositados','ccheques_emitidos','mcheques_emitidos','ccheques_depositados_rechazados','mcheques_depositados_rechazados','ccheques_emitidos_rechazados','mcheques_emitidos_rechazados','ccallcenter_transacciones','chomebanking_transacciones','ccajas_transacciones','ccajas_consultas','ccajas_depositos','ccajas_extracciones','ccajas_otras','catm_trx','matm','catm_trx_other','matm_other','ctrx_quarter','Master_Fvencimiento','Master_msaldototal','Master_msaldopesos','Master_msaldodolares','Master_mconsumospesos','Master_mconsumosdolares','Master_mlimitecompra','Master_madelantopesos','Master_madelantodolares','Master_fultimo_cierre','Master_mpagado','Master_mpagospesos','Master_mpagosdolares','Master_fechaalta','Master_mconsumototal','Master_cconsumos','Master_cadelantosefectivo','Master_mpagominimo','Visa_mfinanciacion_limite','Visa_Fvencimiento','Visa_Finiciomora','Visa_msaldototal','Visa_msaldopesos','Visa_msaldodolares','Visa_mconsumospesos','Visa_mconsumosdolares','Visa_mlimitecompra','Visa_madelantopesos', 'Visa_madelantodolares','Visa_fultimo_cierre','Visa_mpagado','Visa_mpagospesos','Visa_mpagosdolares','Visa_fechaalta','Visa_mconsumotal','Visa_cconsumos','Visa_cadelantosefectivo','Visa_mpagominimo']


In [19]:
# con.execute('''
# CREATE OR REPLACE TABLE df_train AS
# SELECT *,
#     (COALESCE(mpayroll, 0) - COALESCE(mcuentas_saldo, 0)) / NULLIF(ctrx_quarter, 0) AS intensidad_gasto
# FROM df_train
# ''')

In [20]:
# df_train['intensidad_gasto'] = (df_train['mpayroll'].fillna(0) - df_train['mcuentas_saldo'].fillna(0)) / df_train['ctrx_quarter'].replace(0, pd.NA)

In [21]:
# df_train['t_saldo_total'] = df_train['Visa_msaldototal'].fillna(0) + df_train['Master_msaldototal'].fillna(0)

In [22]:
# con.execute('''
# create or replace table df_train as
# select 
#     *,
#     (ifnull(Master_mconsumototal,0) + ifnull(Visa_mconsumototal,0)) as t_consumo_total
# from df_train''')


In [23]:
# con.execute('''
#             create or replace table df_train as
#             select 
#                     *,
#             (IFNULL(t_consumo_total,0) - IFNULL(mpayroll,0)) /NULLIF((IFNULL(mcuentas_saldo, 0) - IFNULL(t_saldo_total,0)), 0) as nivel_endeudamiento, 
#             FROM df_train''')

In [24]:
# df_train['t_consumo_total'] = df_train['Master_mconsumototal'].fillna(0) + df_train['Visa_mconsumototal'].fillna(0)

### Status tarjeta

In [25]:
tarjetas_variables = [
    
    ('ctarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos'),
    ('mttarjeta_visa_debitos_automaticos', 'mttarjeta_master_debitos_automaticos'),
    ('ctarjeta_visa_descuentos', 'ctarjeta_master_descuentos'),
    ('mtarjeta_visa_descuentos', 'mtarjeta_master_descuentos'),
   
    ('Visa_msaldototal', 'Master_msaldototal'),
    ('Visa_msaldopesos', 'Master_msaldopesos'),
    ('Visa_msaldodolares', 'Master_msaldodolares'),
    ('Visa_mconsumospesos', 'Master_mconsumospesos'),
    ('Visa_mconsumosdolares', 'Master_mconsumosdolares'),
    ('Visa_mlimitecompra', 'Master_mlimitecompra'),
    ('Visa_madelantopesos', 'Master_madelantopesos'),
    ('Visa_madelantodolares', 'Master_madelantodolares'),
    ('Visa_fultimo_cierre', 'Master_fultimo_cierre'),
    ('Visa_mpagado', 'Master_mpagado'),
    ('Visa_mpagospesos', 'Master_mpagospesos'),
    ('Visa_mpagosdolares', 'Master_mpagosdolares'),
    ('Visa_fechaalta', 'Master_fechaalta'),
    
    ('Visa_cconsumos', 'Master_cconsumos'),
    ('Visa_cadelantosefectivo', 'Master_cadelantosefectivo'),
    ('Visa_mpagominimo', 'Master_mpagominimo')
]


In [26]:
paired_columns = []
for visa, master in tarjetas_variables:
    paired_columns.append(f"IFNULL({visa}, 0) + IFNULL ({master},0) AS conjunto_{visa}")

paired_columns_sql = ",\n    ".join(paired_columns)





In [27]:
paired_columns

['IFNULL(ctarjeta_visa_debitos_automaticos, 0) + IFNULL (ctarjeta_master_debitos_automaticos,0) AS conjunto_ctarjeta_visa_debitos_automaticos',
 'IFNULL(mttarjeta_visa_debitos_automaticos, 0) + IFNULL (mttarjeta_master_debitos_automaticos,0) AS conjunto_mttarjeta_visa_debitos_automaticos',
 'IFNULL(ctarjeta_visa_descuentos, 0) + IFNULL (ctarjeta_master_descuentos,0) AS conjunto_ctarjeta_visa_descuentos',
 'IFNULL(mtarjeta_visa_descuentos, 0) + IFNULL (mtarjeta_master_descuentos,0) AS conjunto_mtarjeta_visa_descuentos',
 'IFNULL(Visa_msaldototal, 0) + IFNULL (Master_msaldototal,0) AS conjunto_Visa_msaldototal',
 'IFNULL(Visa_msaldopesos, 0) + IFNULL (Master_msaldopesos,0) AS conjunto_Visa_msaldopesos',
 'IFNULL(Visa_msaldodolares, 0) + IFNULL (Master_msaldodolares,0) AS conjunto_Visa_msaldodolares',
 'IFNULL(Visa_mconsumospesos, 0) + IFNULL (Master_mconsumospesos,0) AS conjunto_Visa_mconsumospesos',
 'IFNULL(Visa_mconsumosdolares, 0) + IFNULL (Master_mconsumosdolares,0) AS conjunto_Visa

In [28]:
query = f"""
CREATE OR REPLACE TABLE competencia_03 AS
SELECT *,
    {paired_columns_sql}
FROM competencia_03
"""

%sql {{query}}

,Success


In [29]:
%%sql 


create or replace table competencia_03 as
select
    *,
    (case when Visa_status = 0 then True end) as T_Visa_normal,
    (case when Master_status = 0 then True end) as T_Master_normal
from competencia_03;


create or replace table competencia_03 as
        select *,
        CASE 
        WHEN Visa_status = 6 then 1
        WHEN Visa_status = 7 then 2 
        WHEN Visa_status = 8 then 3
        ELSE NULL
        END as Visa_status
        from competencia_03;

,Success


In [30]:
%%sql

alter table competencia_03 drop column visa_status;
alter table competencia_03 drop column Master_status;

,Success


In [31]:
# df_train['T_Visa_normal'] = df_train['Visa_status'] == 0


# df_train['T_Master_normal'] = df_train['Master_status'] == 0

In [32]:
# con.execute('''
# create or replace table df_train as
# select *
#     , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
#     , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
# from df_train''')

In [33]:
# con.execute('''create or replace table df_train as
#     select *
#     , Master_mpagado+Visa_mpagado as tarjeta_total_pagado
#     from df_train''')

In [34]:
# con.execute('''create or replace table df_train as
#     select *
#     ,IFNULL(tarjeta_total_pagado, 0) / NULLIF(IFNULL(mprestamos_personales,0),0) as actitud_pago
#             from df_train''')

In [35]:
# con.execute('''
# create or replace table df_train as
# select *,
#     lag (actitud_pago, 1) over (
#         partition by numero_de_cliente 
#         order by foto_mes
#     ) as lag_actitud_pago_1,
#     lag (actitud_pago, 2) over (
#         partition by numero_de_cliente 
#         order by foto_mes
#     ) as lag_actitud_pago_2
# from df_train''')

In [36]:
# df_train['T_Visa_normal'] = df_train['Visa_status'] == 0

# # Calcular la nueva columna T_Master_normal
# df_train['T_Master_normal'] = df_train['Master_status'] == 0

### FE lags

In [37]:
# lista_feature = ['active_quarter',
#  'mrentabilidad',
#  'mrentabilidad_annual',
#  'mcuenta_corriente',
#  'mcaja_ahorro',
#  'mcuentas_saldo',
#  'ctarjeta_debito_transacciones',
#  'mautoservicio',
#  'ctarjeta_visa_transacciones',
#  'mtarjeta_visa_consumo',
#  'mtarjeta_master_consumo',
#  'cprestamos_personales',
#  'mprestamos_prendarios',
#  'cprestamos_hipotecarios',
#  'mprestamos_hipotecarios',
#  'cplazo_fijo',
#  'mplazo_fijo_dolares',
#  'mplazo_fijo_pesos',
#  'cpayroll_trx',
#  'mpayroll',
#  'mpayroll2',
#  'cpayroll2_trx',
#  'ccuenta_debitos_automaticos',
#  'mcuenta_debitos_automaticos',
#  'ctarjeta_visa_debitos_automaticos',
#  'mttarjeta_visa_debitos_automaticos',
#  'ctarjeta_master_debitos_automaticos',
#  'mttarjeta_master_debitos_automaticos',
#  'cpagodeservicios',
#  'mpagodeservicios',
#  'mpagomiscuentas',
#  'cpagomiscuentas',
#  'ccajeros_propios_descuentos',
#  'mcajeros_propios_descuentos',
#  'ctarjeta_visa_descuentos',
#  'mtarjeta_visa_descuentos',
#  'ctarjeta_master_descuentos',
#  'mtarjeta_master_descuentos',
#  'ccomisiones_mantenimiento',
#  'mcomisiones_mantenimiento',
#  'ccomisiones_otras',
#  'mcomisiones_otras',
#  'cforex',
#  'cforex_buy',
#  'mforex_buy',
#  'cforex_sell',
#  'mforex_sell',
#  'ctransferencias_recibidas',
#  'mtransferencias_recibidas',
#  'ctransferencias_emitidas',
#  'mtransferencias_emitidas',
#  'cextraccion_autoservicio',
#  'mextraccion_autoservicio',
#  'ccheques_depositados',
#  'mcheques_depositados',
#  'ccheques_emitidos',
#  'mcheques_emitidos',
#  'ccheques_depositados_rechazados',
#  'mcheques_depositados_rechazados',
#  'ccheques_emitidos_rechazados',
#  'mcheques_emitidos_rechazados',
#  'ccallcenter_transacciones',
#  'chomebanking_transacciones',
#  'ccajas_transacciones',
#  'ccajas_consultas',
#  'ccajas_depositos',
#  'ccajas_extracciones',
#  'ccajas_otras',
#  'catm_trx',
#  'matm',
#  'catm_trx_other',
#  'matm_other',
#  'ctrx_quarter',
#  'Master_Fvencimiento',
#  'Master_msaldototal',
#  'Master_msaldopesos',
#  'Master_msaldodolares',
#  'Master_mconsumospesos',
#  'Master_mconsumosdolares',
#  'Master_mlimitecompra',
#  'Master_madelantopesos',
#  'Master_madelantodolares',
#  'Master_fultimo_cierre',
#  'Master_mpagado',
#  'Master_mpagospesos',
#  'Master_mpagosdolares',
#  'Master_fechaalta',
#  'Master_mconsumototal',
#  'Master_cconsumos',
#  'Master_cadelantosefectivo',
#  'Master_mpagominimo',
#  'Visa_mfinanciacion_limite',
#  'Visa_Fvencimiento',
#  'Visa_Finiciomora',
#  'Visa_msaldototal',
#  'Visa_msaldopesos',
#  'Visa_msaldodolares',
#  'Visa_mconsumospesos',
#  'Visa_mconsumosdolares',
#  'Visa_mlimitecompra',
#  'Visa_madelantopesos',
#  'Visa_madelantodolares',
#  'Visa_fultimo_cierre',
#  'Visa_mpagado',
#  'Visa_mpagospesos',
#  'Visa_mpagosdolares',
#  'Visa_fechaalta',
#  'Visa_mconsumototal',
#  'Visa_cconsumos',
#  'Visa_cadelantosefectivo',
#  'Visa_mpagominimo']

In [38]:
lista_feature = ['active_quarter',
 'mrentabilidad',
 
 'mcuenta_corriente',
 'mcaja_ahorro',
 'mcuentas_saldo',
 'ctarjeta_debito_transacciones',
 'mautoservicio',
 'mprestamos_prendarios',
 'cprestamos_hipotecarios',
 'mprestamos_hipotecarios',
 'cplazo_fijo',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'cpayroll_trx',
 'mpayroll',
 'mpayroll2',
 'cpayroll2_trx',
 'ccuenta_debitos_automaticos',
 'mcuenta_debitos_automaticos',
 'cpagodeservicios',
 'mpagodeservicios',
 'mpagomiscuentas',
 'cpagomiscuentas',
 'ccajeros_propios_descuentos',
 'mcajeros_propios_descuentos',
 'ccomisiones_mantenimiento',
 'ccomisiones_otras',
 'cforex',
 'cforex_buy',
 'mforex_buy',
 'cforex_sell',
 'mforex_sell',
 'ctransferencias_recibidas',
 'ctransferencias_emitidas',
 'mtransferencias_emitidas',
 'cextraccion_autoservicio',
 'mextraccion_autoservicio',
 'ccheques_depositados',
 'mcheques_depositados',
 'ccheques_emitidos',
 'mcheques_emitidos',
 'ccheques_depositados_rechazados',
 'mcheques_depositados_rechazados',
 'ccheques_emitidos_rechazados',
 'mcheques_emitidos_rechazados',
 'ccallcenter_transacciones',
 'chomebanking_transacciones',
 'ccajas_transacciones',
 'ccajas_consultas',
 'ccajas_depositos',
 'ccajas_extracciones',
 'ccajas_otras',
 'catm_trx',
 'matm',
 'catm_trx_other',
 'matm_other',
 'ctrx_quarter',
 'conjunto_ctarjeta_visa_debitos_automaticos',
 'conjunto_mttarjeta_visa_debitos_automaticos',
 'conjunto_ctarjeta_visa_descuentos',
 'conjunto_mtarjeta_visa_descuentos',
 'conjunto_Visa_msaldototal',
 'conjunto_Visa_msaldopesos',
 'conjunto_Visa_msaldodolares',
 'conjunto_Visa_mconsumospesos',
 'conjunto_Visa_mconsumosdolares',
 'conjunto_Visa_mlimitecompra',
 'conjunto_Visa_madelantopesos',
 'conjunto_Visa_madelantodolares',
 'conjunto_Visa_fultimo_cierre',
 'conjunto_Visa_mpagado',
 'conjunto_Visa_mpagospesos',
 'conjunto_Visa_mpagosdolares',
 'conjunto_Visa_fechaalta',
 'conjunto_Visa_cconsumos',
 'conjunto_Visa_cadelantosefectivo',
 'conjunto_Visa_mpagominimo']



In [39]:
lag_columns = []
for variable in lista_feature:
    lag_columns.append(f"lag({variable}, 1) over (partition by numero_de_cliente order by foto_mes) as {variable}_lag_1")
    lag_columns.append(f"lag({variable}, 2) over (partition by numero_de_cliente order by foto_mes) as {variable}_lag_2")


lag_columns_sql = ",\n    ".join(lag_columns)


query = f"""
CREATE OR REPLACE TABLE competencia_03 AS
SELECT *,
    {lag_columns_sql}
FROM competencia_03
"""

%sql {{query}}



,Success


In [40]:
delta_columns = []
for variable in lista_feature:
    nombre_variable = variable+'_delta_1'
    nombre_variable_2 = variable+'_delta_2'
    delta_columns.append(f"{variable} - {variable}_lag_1 as {nombre_variable}")
    delta_columns.append(f"{variable}_lag_1 - {variable}_lag_2 as {nombre_variable_2}")

delta_columns_sql = ",\n    ".join(delta_columns)


query = f'''CREATE OR REPLACE TABLE competencia_03 AS
            select *,
            {delta_columns_sql}
            from competencia_03'''
%sql {{query}}
            

,Success


In [41]:
path_dataset_lags_deltas_y_clase_ternaria = datasets_path_l+r'\competencia_03_lags_deltas_y_clase_ternaria.parquet'
path_dataset_lags_deltas_y_clase_ternaria


'C:\\Users\\Admin\\Documents\\1_Notebook\\1_Estudio\\1_UBA_Maestria_DS\\1_Especializacion\\1_Segundo_Semestre\\DMEyF\\datasets\\competencia_3\\competencia_03_lags_deltas_y_clase_ternaria.parquet'

In [42]:
%sql COPY competencia_03 TO '{{path_dataset_lags_deltas_y_clase_ternaria}}' (FORMAT 'parquet')


,Success


In [ ]:
datita = pd.read_parquet('C:\\Users\\Admin\\Documents\\1_Notebook\\1_Estudio\\1_UBA_Maestria_DS\\1_Especializacion\\1_Segundo_Semestre\\DMEyF\\datasets\\competencia_2\\competencia_02_lags_deltas_y_clase_ternaria.parquet'
) 
datita.shape

(4735593, 600)

In [8]:
datita['clase_ternaria']

0          CONTINUA
1          CONTINUA
2          CONTINUA
3          CONTINUA
4          CONTINUA
             ...   
4735588    CONTINUA
4735589    CONTINUA
4735590    CONTINUA
4735591    CONTINUA
4735592    CONTINUA
Name: clase_ternaria, Length: 4735593, dtype: object

In [ ]:

# con.execute('''
# create or replace table df_train as
# select *,
#     lag (mpayroll, 1) over (
#         partition by numero_de_cliente 
#         order by foto_mes
#     ) as lag_mpayroll1,
#     lag (mpayroll, 2) over (
#         partition by numero_de_cliente 
#         order by foto_mes
#     ) as lag_mpayroll2
# from df_train''')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [81]:

# df_sorted = df_train.sort_values(by=['numero_de_cliente', 'foto_mes'])


# df_train['lag_ctrx_quarter1'] = df_sorted.groupby('numero_de_cliente')['ctrx_quarter'].shift(1)


# df_train['lag_ctrx_quarter2'] = df_sorted.groupby('numero_de_cliente')['ctrx_quarter'].shift(2)

In [82]:
# con.execute('''
# create or replace table df_train as
# select * 
#     , lag (t_consumo_total, 1) over (
#         partition by numero_de_cliente 
#         order by foto_mes, numero_de_cliente
#     ) as lag_consumo_total1
#     , lag (t_consumo_total, 2) over (
#         partition by numero_de_cliente 
#         order by foto_mes, numero_de_cliente
#     ) as lag_consumo_total2
# from df_train''')

### FE Deltas

In [ ]:
# con.execute('''create or replace table df_train as
# select *
#   , ctrx_quarter - lag_ctrx_quarter1 as delta_1_ctrx_quarter
#   , ctrx_quarter - lag_ctrx_quarter2 as delta_1_ctrx_quarter
# from df_train''')

In [84]:

# con.execute('''create or replace table df_train as
# select *
#   , t_consumo_total - lag_consumo_total1 as delta_1_lag_confumo_total
#   , t_consumo_total - lag_consumo_total2 as delta_2_lag_consumo_total
# from df_train''')

In [85]:
# con.execute(''' create or replace table df_train as
# select *
#   ,         IFNULL (t_consumo_total,0) / NULLIF (IFNULL (mpayroll,0),0) as consumo_vs_payroll
#             from df_train''')

In [86]:
# con.execute('''
# create or replace table df_train as
# select * 
#     , lag (consumo_vs_payroll, 1) over (
#         partition by numero_de_cliente 
#         order by foto_mes, numero_de_cliente
#     ) as lag_consumo_vs_payroll_1
#     , lag (consumo_vs_payroll, 2) over (
#         partition by numero_de_cliente 
#         order by foto_mes, numero_de_cliente
#     ) as lag_consumo_vs_payroll_2
# from df_train''')

In [87]:
# # Ordenar el DataFrame por 'numero_de_cliente' y 'foto_mes'
# df_sorted = df_train.sort_values(by=['numero_de_cliente', 'foto_mes'])

# # Calcular la columna 'lag_consumo_total1'
# df_train['lag_consumo_total1'] = df_sorted.groupby('numero_de_cliente')['t_consumo_total'].shift(1)

# # Calcular la columna 'lag_consumo_total2'
# df_train['lag_consumo_total2'] = df_sorted.groupby('numero_de_cliente')['t_consumo_total'].shift(2)

### FE Percents

In [ ]:
# for variable in lista_features:
        
#     con.execute('''
                
        
#     create or replace table df_train as
#     select *
#         , percent_rank() over (partition by foto_mes order by {variable},numero_de_cliente) as cliente_antiguedad_dd
#          as {nombre_variable}
#     from df_train''')



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [89]:
# con.execute('''create or replace table df_train as
# select *, 
#             greatest(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as max_tarjeta_transacciones,
#             greatest(ccallcenter_transacciones, chomebanking_transacciones) as max_dig_analog_transacciones,
#             least(ctarjeta_debito_transacciones, ctarjeta_visa_transacciones) as min_tarjeta_debito_transacciones,
#             least(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as min_tarjeta_master_transacciones,
#             least(ccallcenter_transacciones, chomebanking_transacciones) as min_callcenter_transacciones,
#             from df_train''')
            

In [90]:
# # Calcular percent_rank para cada columna
# df_train['cliente_antiguedad_dd'] = df_train.groupby('foto_mes')['cliente_antiguedad'].rank(method='min', pct=True)
# df_train['cliente_edad_dd'] = df_train.groupby('foto_mes')['cliente_edad'].rank(method='min', pct=True)
# df_train['ctrx_quarter_dd'] = df_train.groupby('foto_mes')['ctrx_quarter'].rank(method='min', pct=True)
# df_train['mactivos_margen_dd'] = df_train.groupby('foto_mes')['mactivos_margen'].rank(method='min', pct=True)
# df_train['mpayroll_dd'] = df_train.groupby('foto_mes')['mpayroll'].rank(method='min', pct=True)
# df_train['intensidad_gasto_dd'] = df_train.groupby('foto_mes')['intensidad_gasto'].rank(method='min', pct=True)
# df_train['cproductos_dd'] = df_train.groupby('foto_mes')['cproductos'].rank(method='min', pct=True)

In [91]:
# df_train = con.execute('''SELECT * FROM df_train''').fetch_df()


In [92]:
# con.close()

In [93]:
# df_train.shape

In [94]:
# # df_train.to_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_03_f_e.csv", index=False)
# df_train = pd.read_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_03_f_e.csv", index_col=False)    

### Modelo para variables más importantes + optimización 

In [96]:
# optuna.visualization.plot_optimization_history(study)

In [97]:
# plot_param_importances(study)

In [98]:
# plot_slice(study)

In [99]:
# best_params = study.best_params
# print("Mejores parámetros:", best_params)